In [ ]:
%%time
# for development, changes in other modules have to be reloaded to reflect changes
%load_ext autoreload
%autoreload 1

import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sbn 
import os, sys
import SimpleITK as sitk

from scipy.ndimage.filters import gaussian_filter

%aimport config 
sys.path.append(config.lib_dir)
%aimport utils
%aimport segment 
%aimport match 
%aimport register 
%aimport evaluate 
%aimport qc 

In [ ]:
?sitk.RescaleIntensity

In [ ]:
_dir = !pwd
print('this dir:', _dir[0])
print('data dir:', config.data_dir) 
print('output dir:', config.output_dir)
print('slide name:', config.slide_name)
print('scene:', config.scene_name)

# Overview 

In [ ]:
img_file_names = [x for x in os.listdir(config.data_dir) if x[-4:] == '.tif']
parsed_names = pd.DataFrame([utils.parse_file_name(x) for x in img_file_names])
parsed_names.head()

In [ ]:
parsed_names2 = parsed_names[parsed_names.slide_name == config.slide_name]
print('sanity check - scenes:', parsed_names2['scene'].unique())
parsed_names3 = parsed_names2[parsed_names2.scene == config.scene_name]

# Load images 

This can take a few (>30) minutes, grab a cup of tea. 

In [ ]:
%%time
fs = [x for x in parsed_names3.original.values if 'c1' in x]
imgs = utils.load_imgs_mt(fs, config.data_dir, _type=sitk.sitkUInt16)

## Grab two dapi images from different rounds

In [ ]:
k = list(imgs.keys())
k.sort()
k

In [ ]:
img1 = imgs['R5_BCLxL.CD68.PD1.pATM_D1_2020_10_08__9273_c1_ORG.tif']
img2 = imgs['R4_pATR.CCNB1.CD4.53BP1_D1_2020_10_07__9253_c1_ORG.tif']

In [ ]:
utils.myshow(img1[::10, ::10])

In [ ]:
utils.myshow(img2[::10, ::10])

# if we assume a significant left-right shift only 

In [ ]:
img1_arr = sitk.GetArrayFromImage(img1)
arr1 = np.log10(np.sum(img1_arr, axis=0) + 1) # take column sums - add one to avoid log10(0)
arr1 = (arr1 - arr1.mean())/arr1.std() # zscore to try and normalize - adjust for differences in intensities

img2_arr = sitk.GetArrayFromImage(img2)
arr2 = np.log10(np.sum(img2_arr, axis=0) + 1) # take column sums - add one to avoid log10(0)
arr2 = (arr2 - arr2.mean())/arr2.std() # zscore to try and normalize - adjust for differences in intensities

plt.figure(figsize=(15,5))
plt.plot(arr1, c='r', label='img1')
plt.plot(arr2, c='b', label='img2')
plt.xlabel('img x-pixel position')
plt.ylabel('summed signal')
plt.legend()
plt.show()

In [ ]:
# negative shift (e.g. shift image 2 left)
cross_cor = np.correlate(arr1, arr2, 'full')

full_shift = np.concatenate((np.arange(-arr1.shape[0], -1, 1), np.arange(0,arr2.shape[0],1)))

_best_shift = full_shift[cross_cor == np.max(cross_cor)][0]
print(_best_shift)

plt.figure()
plt.plot(full_shift, cross_cor)
plt.xlabel('img2 shift')
plt.ylabel('cross correllation')
plt.axvline(_best_shift, c='r')
plt.show()
    

In [ ]:
def alignment_1d(fixed, moving, axis='x'): 
    '''
    This method sums [x,y] 1 axis and transforms the data as zscore( log10(1+sum(axis)) ). 
    We then use cross correllation to test the optimal shift +/-. 
    The best shift is returned, previously non-existant pixels are introduced as zero values. 

    input
        fixed   sitk.image  the image to align to 
        moving  sitk.image  the image to align
        axis    str         the axis to align, can be 'x' or 'y'

    output 
        int                 optimal number of pxiels to shift 
    '''

    assert axis in ['x', 'y'], 'unrecognized axis type, can be either "x" or "y"'
    _axis={'x':0, 'y':1}

    def get_1d_arr(img):
        img_arr = sitk.GetArrayFromImage(img)
        arr1 = np.log10(np.sum(img_arr, axis=_axis[axis]) + 1) # take column sums - add one to avoid log10(0)
        arr1 = (arr1 - arr1.mean())/arr1.std() # zscore to try and normalize - adjust for differences in intensities
        return arr1

    fixed_arr = get_1d_arr(fixed)
    moving_arr = get_1d_arr(moving)

    cross_cor = np.correlate(fixed_arr,moving_arr, 'full')
    
    full_shift = np.concatenate((np.arange(-moving_arr.shape[0], -1, 1), np.arange(0,fixed_arr.shape[0],1)))
    _best_shift = full_shift[cross_cor == np.max(cross_cor)][0]

    return _best_shift

In [ ]:
def shift_image(img, ref, x, y): 
    '''
    performs the resampling to shift the image properly. Can also be used to crop/expand to proper size. 

    input 
        img     sitk.Image      image to shift
        ref
        x       int             pixels to shift, can be negative or positive 
        y       int             pixels to shift, can be negative or positive 

    output 
        sitk.Image              shifted image
    '''
    translation = sitk.TranslationTransform(img.GetDimension())
    x,y = float(x), float(y)
    translation.SetOffset((-x,-y))
    return sitk.Resample(img,
                         ref, 
                         translation,
                         sitk.sitkLinear,
                         0)

In [ ]:
img1 = imgs['R5_BCLxL.CD68.PD1.pATM_D1_2020_10_08__9273_c1_ORG.tif']
img2 = imgs['R4_pATR.CCNB1.CD4.53BP1_D1_2020_10_07__9253_c1_ORG.tif']

In [ ]:
x = alignment_1d(img1, img2, axis='x')
y = alignment_1d(img1, img2, axis='y')
print(x,y)
print(type(x))

In [ ]:
img2_shifted =shift_image(img2, img1, x, y)

In [ ]:
print(img1.GetSize())
print(img2_shifted.GetSize())

In [ ]:
#downsample 
ds = 10
img1_ds = img1[::ds, ::ds]# > 1
img2_ds = img2_shifted[::ds, ::ds]# > 1

# cast as float to avoid int overflow 
img1_ds = sitk.Cast(img1_ds, sitk.sitkFloat32)
img2_ds = sitk.Cast(img2_ds, sitk.sitkFloat32)

# take differnce
sub = img1_ds - img2_ds

# plot difference
utils.myshow(sub, figsize=(10,10))

In [ ]:
#downsample 
ds = 10
img1_ds = img1[::ds, ::ds] > 1
img2_ds = img2_shifted[::ds, ::ds] > 1


# cast as float to avoid int overflow 
img1_ds = sitk.Cast(img1_ds, sitk.sitkFloat32)
img2_ds = sitk.Cast(img2_ds, sitk.sitkFloat32)

# take differnce
sub = img1_ds - img2_ds

# plot difference
utils.myshow(sub, figsize=(10,10))

In [ ]:
sitk.RescaleIntensity()